In [77]:
import pandas as pd
# Setze die maximale Anzahl an Spalten, die angezeigt werden sollen
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib as plt
import os

In [78]:
# Print Article Content
with open(".\\training\\12", "r", encoding="utf-8") as file:
    inhalt = file.read()
#print(inhalt)

In [79]:
def CreateFileList(files_path):
    list_of_files = []
    
    for file in os.listdir(files_path):
        list_of_files.append(file)
    
    return list_of_files

file_list_train = CreateFileList(".\\training\\")
file_list_test = CreateFileList(".\\test\\")

In [80]:
train_df = pd.read_csv("ModApte_train.csv")
test_df = pd.read_csv("ModApte_test.csv")
# remove " symbol from new_id column
train_df["new_id"] = train_df["new_id"].str.replace('"','',regex=False)
test_df["new_id"] = test_df["new_id"].str.replace('"','',regex=False)
print(train_df.shape)
train_df.head()

(9603, 13)


,text,text_type,topics,lewis_split,cgis_split,old_id,new_id,places,people,orgs,exchanges,date,title
0,Showers continued throughout the week in\r\nth...,"""NORM""",['cocoa'],"""TRAIN""","""TRAINING-SET""","""5544""",1,['el-salvador' 'usa' 'uruguay'],[],[],[],26-FEB-1987 15:01:01.79,BAHIA COCOA REVIEW
1,The U.S. Agriculture Department\r\nreported th...,"""NORM""",['grain' 'wheat' 'corn' 'barley' 'oat' 'sorghum'],"""TRAIN""","""TRAINING-SET""","""5548""",5,['usa'],[],[],[],26-FEB-1987 15:10:44.60,NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESERVE
2,Argentine grain board figures show\r\ncrop reg...,"""NORM""",['veg-oil' 'linseed' 'lin-oil' 'soy-oil' 'sun-...,"""TRAIN""","""TRAINING-SET""","""5549""",6,['argentina'],[],[],[],26-FEB-1987 15:14:36.41,ARGENTINE 1986/87 GRAIN/OILSEED REGISTRATIONS
3,Moody's Investors Service Inc said it\r\nlower...,"""NORM""",[],"""TRAIN""","""TRAINING-SET""","""5551""",8,['usa'],[],[],[],26-FEB-1987 15:15:40.12,USX &lt;X> DEBT DOWGRADED BY MOODY'S
4,Champion Products Inc said its\r\nboard of dir...,"""NORM""",['earn'],"""TRAIN""","""TRAINING-SET""","""5552""",9,['usa'],[],[],[],26-FEB-1987 15:17:11.20,CHAMPION PRODUCTS &lt;CH> APPROVES STOCK SPLIT


In [81]:
train_df["text_type"].unique()

array(['"NORM"', '"BRIEF"', '"UNPROC"'], dtype=object)

In [82]:
# Clean out entries of csv, that are not available in original data
train_df = train_df.loc[train_df["new_id"].isin(file_list_train)]
test_df = test_df.loc[test_df["new_id"].isin(file_list_test)]
# Remove lewis_split and cgis_split
train_df = train_df.drop(columns=["lewis_split","cgis_split","old_id","people","orgs","exchanges","text_type"], axis=1)
test_df = test_df.drop(columns=["lewis_split","cgis_split","old_id","people","orgs","exchanges","text_type"], axis=1)

In [83]:
# Day Month and Year as own features
def extract_date_components(row):
    months = {
    "JAN":1,
    "FEB":2,
    "MAR":3,
    "APR":4,
    "MAY":5,
    "JUN":6,
    "JUL":7,
    "AUG":8,
    "SEP":9,
    "OCT":10,
    "NOV":11,
    "DEC":12
    }
    date_parts = row['date'].split("-")

    day = int(date_parts[0].strip())
    month = int(months[date_parts[1].strip()])
    year = int(date_parts[2][:4].strip())
    return pd.Series([day, month, year])

train_df[['day', 'month', 'year']] = train_df.apply(extract_date_components, axis=1)
test_df[['day', 'month', 'year']] = test_df.apply(extract_date_components, axis=1)
# Remove original Date column
train_df = train_df.drop(columns=["date"], axis=1)
test_df = test_df.drop(columns=["date"], axis=1)

In [84]:
# Always take first topic of article
def one_topic(df):
    topic_list = []
    for topics in df["topics"]:
        topic = topics[2:].split("'",2)[0]
        topic_list.append(topic)
    df["topics"] = topic_list
    return df

train_df = one_topic(train_df)
test_df = one_topic(test_df)


In [85]:
train_df.shape

(7769, 8)

In [86]:
def places_encoding(df):
    unique_places = []
    
    # Get all places
    for place in df["places"]:
        place_list = place.replace("'", "").replace("[", "").replace("]", "").split()
        try:
            for cur_place in place_list:
                if cur_place in unique_places:
                    continue
                else:
                    unique_places.append(cur_place)
                    df[cur_place] = 0
        except:
            continue

    return df

def apply_one_hot_places(row):
    # Greifen Sie auf den gesamten String in row["places"] zu, ohne [0]
    places = row["places"].replace("'", "").replace("[", "").replace("]", "").split()
    
    # Setzen Sie die One-Hot-Encoded-Werte
    for place in places:
        if place in row:
            row[place] = 1
    return row

train_df = places_encoding(train_df)
train_df = train_df.apply(apply_one_hot_places, axis=1)
test_df = places_encoding(test_df)
test_df = test_df.apply(apply_one_hot_places, axis=1)


C:\Users\tyron\AppData\Local\Temp\ipykernel_12488\554344239.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[cur_place] = 0
C:\Users\tyron\AppData\Local\Temp\ipykernel_12488\554344239.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[cur_place] = 0
C:\Users\tyron\AppData\Local\Temp\ipykernel_12488\554344239.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=

In [87]:
train_df.head()

,text,topics,new_id,places,title,day,month,year,el-salvador,usa,uruguay,argentina,yemen-arab-republic,canada,uk,brazil,new-zealand,panama,honduras,philippines,china,algeria,switzerland,ussr,france,japan,australia,thailand,colombia,italy,indonesia,kuwait,ecuador,bahrain,saudi-arabia,qatar,oman,uae,egypt,zambia,india,netherlands,hong-kong,sweden,south-africa,malaysia,zaire,bolivia,nigeria,south-korea,taiwan,pakistan,belgium,west-germany,bangladesh,bhutan,nepal,yemen-demo-republic,iran,iraq,turkey,spain,austria,liechtenstein,cuba,syria,portugal,greece,finland,mexico,zimbabwe,ireland,kenya,poland,czechoslovakia,romania,yugoslavia,chile,liberia,denmark,peru,morocco,dominican-republic,sri-lanka,guatemala,costa-rica,israel,ivory-coast,singapore,luxembourg,papua-new-guinea,gabon,libya,venezuela,jordan,east-germany,jamaica,ghana,nicaragua,paraguay,uganda,hungary,kampuchea,sudan,burma,norway,haiti,aruba,tanzania,cyprus,madagascar,ethiopia,tunisia,suriname,iceland,trinidad-tobago,north-korea,bermuda,vietnam,botswana,cameroon,togo,lebanon,mauritius,mauritania,malawi,mozambique,vanuatu,bulgaria,malta
0,Showers continued throughout the week in\r\nth...,cocoa,1,['el-salvador' 'usa' 'uruguay'],BAHIA COCOA REVIEW,26,2,1987,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The U.S. Agriculture Department\r\nreported th...,grain,5,['usa'],NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESERVE,26,2,1987,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Argentine grain board figures show\r\ncrop reg...,veg-oil,6,['argentina'],ARGENTINE 1986/87 GRAIN/OILSEED REGISTRATIONS,26,2,1987,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Champion Products Inc said its\r\nboard of dir...,earn,9,['usa'],CHAMPION PRODUCTS &lt;CH> APPROVES STOCK SPLIT,26,2,1987,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Computer Terminal Systems Inc said\r\nit has c...,acq,10,['usa'],COMPUTER TERMINAL SYSTEMS &lt;CPML> COMPLETES ...,26,2,1987,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [88]:
train_df.isnull().sum()
# Remove all entries, where text or title is null
train_df = train_df.dropna(subset=['text', 'title'])
test_df = test_df.dropna(subset=['text', 'title'])

In [89]:
for col in train_df.columns:
    if train_df[col].dtype == "object":
        print(col)

text
topics
new_id
places
title


In [90]:
# ID Liste erstellen
train_ids = train_df["new_id"]
test_ids = test_df["new_id"]
train_df = train_df.drop(columns=["new_id","places"], axis=1)
test_df = test_df.drop(columns=["new_id","places"], axis=1)

In [91]:
from sklearn.preprocessing import LabelEncoder
label_encoders = {}

for col in train_df.columns:
    if train_df[col].dtype == "object" and col!="text" and col!="title":
        # Initialisiere den LabelEncoder
        le = LabelEncoder()
        # Fitte und transformiere die Spalte
        train_df[col] = le.fit_transform(train_df[col])
        # Speichere den LabelEncoder für die Spalte
        label_encoders[col] = le

In [92]:
train_df["topics"].unique()

array([ 6, 22, 71, 16,  0, 72,  9, 26,  8, 66, 70, 55, 60, 41, 11,  5, 14,
       42, 12, 40, 29, 20, 63, 39,  1, 68, 45, 21, 69, 65, 37, 56, 30, 31,
       58, 52, 24, 33, 36,  4, 74, 46, 48, 19, 61, 73, 17, 25, 15, 38, 67,
       28, 35, 51, 34, 57, 43, 13, 10, 18, 32, 27, 53, 49, 59, 44, 47,  7,
       54, 64, 23,  2, 62, 50,  3])

In [93]:
def get_original_topic(label_encoders, column_name, number):
    if column_name in label_encoders:
        le = label_encoders[column_name]
        try:
            # Wandelt die Zahl in das ursprüngliche Topic um
            original_topic = le.inverse_transform([number])[0]
            return original_topic
        except IndexError:
            return "Ungültige Zahl: Diese Zahl ist nicht im LabelEncoder enthalten."
    else:
        return "LabelEncoder für diese Spalte wurde nicht gefunden."

# Beispielnutzung
topic = get_original_topic(label_encoders, 'topics', 6)
print(topic)

cocoa


In [94]:
train_df.head()

,text,topics,title,day,month,year,el-salvador,usa,uruguay,argentina,yemen-arab-republic,canada,uk,brazil,new-zealand,panama,honduras,philippines,china,algeria,switzerland,ussr,france,japan,australia,thailand,colombia,italy,indonesia,kuwait,ecuador,bahrain,saudi-arabia,qatar,oman,uae,egypt,zambia,india,netherlands,hong-kong,sweden,south-africa,malaysia,zaire,bolivia,nigeria,south-korea,taiwan,pakistan,belgium,west-germany,bangladesh,bhutan,nepal,yemen-demo-republic,iran,iraq,turkey,spain,austria,liechtenstein,cuba,syria,portugal,greece,finland,mexico,zimbabwe,ireland,kenya,poland,czechoslovakia,romania,yugoslavia,chile,liberia,denmark,peru,morocco,dominican-republic,sri-lanka,guatemala,costa-rica,israel,ivory-coast,singapore,luxembourg,papua-new-guinea,gabon,libya,venezuela,jordan,east-germany,jamaica,ghana,nicaragua,paraguay,uganda,hungary,kampuchea,sudan,burma,norway,haiti,aruba,tanzania,cyprus,madagascar,ethiopia,tunisia,suriname,iceland,trinidad-tobago,north-korea,bermuda,vietnam,botswana,cameroon,togo,lebanon,mauritius,mauritania,malawi,mozambique,vanuatu,bulgaria,malta
0,Showers continued throughout the week in\r\nth...,6,BAHIA COCOA REVIEW,26,2,1987,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The U.S. Agriculture Department\r\nreported th...,22,NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESERVE,26,2,1987,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Argentine grain board figures show\r\ncrop reg...,71,ARGENTINE 1986/87 GRAIN/OILSEED REGISTRATIONS,26,2,1987,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Champion Products Inc said its\r\nboard of dir...,16,CHAMPION PRODUCTS &lt;CH> APPROVES STOCK SPLIT,26,2,1987,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Computer Terminal Systems Inc said\r\nit has c...,0,COMPUTER TERMINAL SYSTEMS &lt;CPML> COMPLETES ...,26,2,1987,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [95]:
train_df_test = train_df[:100]

In [ ]:
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Load pre-trained model
model = BertModel.from_pretrained('bert-base-uncased')

def get_embeddings_batch(texts, batch_size=8):
    # Liste zur Speicherung der Embeddings
    all_embeddings = []

    # Erstelle einen DataLoader für die Batch-Verarbeitung
    text_loader = DataLoader(texts, batch_size=batch_size, num_workers=4)

    for batch in text_loader:
        # Tokenisiere und kodiere den Batch von Texten
        encoded_input = tokenizer(
            batch,
            max_length=512,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        # Berechne die Embeddings mit dem BERT-Modell
        with torch.no_grad():
            outputs = model(**encoded_input)

        # Extrahiere die Embeddings und füge sie zur Liste hinzu
        embeddings = outputs.last_hidden_state
        all_embeddings.append(embeddings)

    # Kombiniere alle Embeddings zu einem Tensor
    all_embeddings = torch.cat(all_embeddings, dim=0)
    return all_embeddings

embeddings = get_embeddings_batch(train_df_test["text"].tolist())

In [97]:
# shape of embeddings
embeddings.shape

torch.Size([100, 512, 768])